In [1]:
import sys
sys.path.insert(1, '../')
from FrenetFDA.processing_Euclidean_curve_bis.unified_estimate_Frenet_state_space.EM_Frenet_state_space import FrenetStateSpace
from FrenetFDA.processing_Euclidean_curve_bis.unified_estimate_Frenet_state_space.iek_filter_smoother_Frenet_state import IEKFilterSmootherFrenetState
import FrenetFDA.utils.visualization as visu
from FrenetFDA.utils.Lie_group.SE3_utils import SE3
from FrenetFDA.utils.Lie_group.SO3_utils import SO3
from FrenetFDA.utils.smoothing_utils import VectorBSplineSmoothing
from FrenetFDA.utils.Frenet_Serret_utils import solve_FrenetSerret_ODE_SE, solve_FrenetSerret_ODE_SO
import numpy as np
from scipy.integrate import cumtrapz, solve_ivp
from scipy.interpolate import interp1d
from sklearn.gaussian_process.kernels import Matern
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
layout = go.Layout(plot_bgcolor='rgba(0,0,0,0)')
import plotly.io as pio
pio.renderers.default = 'browser'

INFO: Using numpy backend


In [2]:
def generate_Y(X, gamma):
    W = gamma**2*np.eye(3)
    Y = X[1:] + np.random.multivariate_normal(np.zeros(3), W, size=(len(X)-1))
    return Y

color_list = px.colors.qualitative.Plotly
def plot_res_Z(kf, Q_true, Q_mu, X_true, X_mu, Y, title=''):

    fig = make_subplots(rows=2, cols=2, subplot_titles=("Tangent Vector", "Normal Vector", "Binormal Vector", "X"), specs=[[{"type": "scene"}, {"type": "scene"}],
           [{"type": "scene"}, {"type": "scene"}]],)
    # T 
    fig.add_trace(go.Scatter3d(x=Q_true[:,:,0][:,0],y=Q_true[:,:,0][:,1],z=Q_true[:,:,0][:,2],name='True',mode='lines',line=dict(width=3,  color=color_list[0])), row=1, col=1)
    fig.add_trace(go.Scatter3d(x=Q_mu[:,:,0][:,0],y=Q_mu[:,:,0][:,1],z=Q_mu[:,:,0][:,2],name='Init',mode='lines',line=dict(width=3,  color=color_list[1])), row=1, col=1)
    fig.add_trace(go.Scatter3d(x=kf.track_Q[:,:,0][:,0],y=kf.track_Q[:,:,0][:,1],z=kf.track_Q[:,:,0][:,2],name='Track',mode='lines',line=dict(width=3,  color=color_list[2])), row=1, col=1)
    fig.add_trace(go.Scatter3d(x=kf.smooth_Q[:,:,0][:,0],y=kf.smooth_Q[:,:,0][:,1],z=kf.smooth_Q[:,:,0][:,2],name='Smooth',mode='lines',line=dict(width=3,  color=color_list[3])), row=1, col=1)
    # N 
    fig.add_trace(go.Scatter3d(x=Q_true[:,:,1][:,0],y=Q_true[:,:,1][:,1],z=Q_true[:,:,1][:,2],name='True',mode='lines',line=dict(width=3,  color=color_list[0])), row=1, col=2)
    fig.add_trace(go.Scatter3d(x=Q_mu[:,:,1][:,0],y=Q_mu[:,:,1][:,1],z=Q_mu[:,:,1][:,2],name='Init',mode='lines',line=dict(width=3, color=color_list[1])), row=1, col=2)
    fig.add_trace(go.Scatter3d(x=kf.track_Q[:,:,1][:,0],y=kf.track_Q[:,:,1][:,1],z=kf.track_Q[:,:,1][:,2],name='Track',mode='lines',line=dict(width=3, color=color_list[2])), row=1, col=2)
    fig.add_trace(go.Scatter3d(x=kf.smooth_Q[:,:,1][:,0],y=kf.smooth_Q[:,:,1][:,1],z=kf.smooth_Q[:,:,1][:,2],name='Smooth',mode='lines',line=dict(width=3,  color=color_list[3])), row=1, col=2)
    # B
    fig.add_trace(go.Scatter3d(x=Q_true[:,:,2][:,0],y=Q_true[:,:,2][:,1],z=Q_true[:,:,2][:,2],name='True',mode='lines',line=dict(width=3, color=color_list[0])), row=2, col=1)
    fig.add_trace(go.Scatter3d(x=Q_mu[:,:,2][:,0],y=Q_mu[:,:,2][:,1],z=Q_mu[:,:,2][:,2],name='Init',mode='lines',line=dict(width=3, color=color_list[1])), row=2, col=1)
    fig.add_trace(go.Scatter3d(x=kf.track_Q[:,:,2][:,0],y=kf.track_Q[:,:,2][:,1],z=kf.track_Q[:,:,2][:,2],name='Track',mode='lines',line=dict(width=3, color=color_list[2])), row=2, col=1)
    fig.add_trace(go.Scatter3d(x=kf.smooth_Q[:,:,2][:,0],y=kf.smooth_Q[:,:,2][:,1],z=kf.smooth_Q[:,:,2][:,2],name='Smooth',mode='lines',line=dict(width=3, color=color_list[3])), row=2, col=1)
    # X 
    fig.add_trace(go.Scatter3d(x=X_true[:,0],y=X_true[:,1],z=X_true[:,2],name='True',mode='lines',line=dict(width=3,  color=color_list[0])), row=2, col=2)
    fig.add_trace(go.Scatter3d(x=X_mu[:,0],y=X_mu[:,1],z=X_mu[:,2],name='Init',mode='lines',line=dict(width=3, color=color_list[1])), row=2, col=2)
    fig.add_trace(go.Scatter3d(x=kf.track_X[:,0],y=kf.track_X[:,1],z=kf.track_X[:,2],name='Track',mode='lines',line=dict(width=3, color=color_list[2])), row=2, col=2)
    fig.add_trace(go.Scatter3d(x=kf.smooth_X[:,0],y=kf.smooth_X[:,1],z=kf.smooth_X[:,2],name='Smooth',mode='lines',line=dict(width=3, color=color_list[3])), row=2, col=2)
    fig.add_trace(go.Scatter3d(x=Y[:,0],y=Y[:,1],z=Y[:,2],name='Observations',mode='lines',line=dict(width=3, color=color_list[4])), row=2, col=2)

    fig.update_xaxes(showline=True, showgrid=False, linewidth=1, linecolor='black')
    fig.update_yaxes(showline=True, showgrid=False, linewidth=1, linecolor='black')
    fig.update_layout(title_text=title)
    fig.show()

N = 200
grid = np.linspace(0,1,N+1)


""" __________ Definition of exact function Theta ___________ """

curv = lambda s : 2*np.cos(2*np.pi*s) + 5
tors = lambda s : 2*np.sin(2*np.pi*s) + 1

def theta(s):
    if isinstance(s, int) or isinstance(s, float):
        return np.array([curv(s), tors(s)])
    elif isinstance(s, np.ndarray):
        return np.vstack((curv(s), tors(s))).T
    else:
        raise ValueError('Variable is not a float, a int or a NumPy array.')
    
def warping(s,a):
    if np.abs(a)<1e-15:
        return s
    else:
        return (np.exp(a*s) - 1)/(np.exp(a) - 1)
    
warp_grid = warping(grid, 2)
    
""" __________ Definition of states and noisy observations __________ """

mu0 = np.eye(4)
mu_Z = solve_FrenetSerret_ODE_SE(theta, grid, Z0=mu0)
mu_Q, mu_X = mu_Z[:,:3,:3], mu_Z[:,:3,3]

gamma = 0.001
W = gamma**2*np.eye(3)
Y = generate_Y(mu_X, gamma)

In [3]:
visu.plot_3D([mu_X, Y], ['X', 'Y'])

In [3]:
# Bspline_true_theta = VectorBSplineSmoothing(2, 30)
# Bspline_true_theta.fit(grid, theta(grid))
# true_coefs = Bspline_true_theta.coefs
sigma_theta = 2
val_theta_noisy = theta(grid) + np.random.multivariate_normal(np.zeros(2), sigma_theta**2*np.eye(2), size=len(grid))
Bspline_noisy_theta = VectorBSplineSmoothing(2, 30)

In [4]:
Bspline_noisy_theta.fit(grid, val_theta_noisy, regularization_parameter=np.array([0.01]))
coefs_noisy = Bspline_noisy_theta.coefs

In [5]:
visu.plot_array_2D_names(grid, [curv(grid), val_theta_noisy[:,0], Bspline_noisy_theta.evaluate(grid)[:,0]], ['exact_function', 'noisy initial guess', 'Bspline representation initial guess'])
visu.plot_array_2D_names(grid, [tors(grid), val_theta_noisy[:,1], Bspline_noisy_theta.evaluate(grid)[:,1]], ['exact_function', 'noisy initial guess', 'Bspline representation initial guess'])

In [5]:
theta_noisy = Bspline_noisy_theta.evaluate

In [6]:
# sigma = 0.02
# Sigma = lambda s: sigma**2*np.eye(2)
sigma_1 = 0.024624036788131774
sigma_2 = 0.031145642946630775
Sigma = lambda s: np.array([[sigma_1**2, 0], [0, sigma_2**2]])

P0 = (0.0001**2)*np.eye(6)

kf = IEKFilterSmootherFrenetState(3, W, Sigma, theta_noisy, Z0=mu0, P0=P0, K_pts=5) 
kf.smoothing(grid,Y)

In [7]:
mu_Z_noisy = solve_FrenetSerret_ODE_SE(theta_noisy, grid, Z0=mu0)
mu_Q_noisy, mu_X_noisy = mu_Z_noisy[:,:3,:3], mu_Z_noisy[:,:3,3]
plot_res_Z(kf, mu_Q, mu_Q_noisy, mu_X, mu_X_noisy, Y, title='test tracking from wrong a_theta')

In [7]:
FS_statespace = FrenetStateSpace(grid,Y)
reg_param_list = np.array([[0.001,0.01], [0.001,0.001], [0.0001,0.001], [0.0001,0.0001], [0.00001,0.00001], [0.000001,0.000001]])
# reg_param_list = np.array([[0.0001,0.0001]])
FS_statespace.initialized_from_state(kf.smooth_Z, coefs_noisy, 30, reg_param_list, P=kf.smooth_P, P_dble=kf.smooth_P_dble, method='autre', model_Sigma='diagonal_continuous')

___ M step ___
iteration for optimization of coefs and Sigma: 0 , relative error: 0.057146457799571386
iteration for optimization of coefs and Sigma: 1 , relative error: 0.005077864772270491
iteration for optimization of coefs and Sigma: 2 , relative error: 0.0007999674711607682
part 1 SSE [0.19602411 1.96884126]
lambda value: [0.001 0.01 ]  GCV score: 0.011732323748954368 SSE: [0.19602411 1.96884126] df: [13.39553065  7.32044805]
iteration for optimization of coefs and Sigma: 0 , relative error: 0.06288353051923813
iteration for optimization of coefs and Sigma: 1 , relative error: 0.001383096610773874
iteration for optimization of coefs and Sigma: 2 , relative error: 1.0862653012741554e-05
part 1 SSE [0.19601607 0.04591444]
lambda value: [0.001 0.001]  GCV score: 0.0013858815782535384 SSE: [0.19601607 0.04591444] df: [13.39553071 12.08120646]
iteration for optimization of coefs and Sigma: 0 , relative error: 0.06309428066674935
iteration for optimization of coefs and Sigma: 1 , relati

In [11]:
FS_statespace.compute_sampling_variance()

part 1 SSE [0.01637076 0.0035135 ]


In [12]:
lower_bound, upper_bound = FS_statespace.confidence_limits(grid)
visu.plot_array_2D_names(grid, [curv(grid), FS_statespace.theta(grid)[:,0], theta_noisy(grid)[:,0], lower_bound[:,0], upper_bound[:,0]], ['curv exact', 'estimated curv', 'curv init guess', 'lower_bound', 'upper_bound'])
visu.plot_array_2D_names(grid, [tors(grid), FS_statespace.theta(grid)[:,1],  theta_noisy(grid)[:,1], lower_bound[:,1], upper_bound[:,1]], ['tors exact', 'estimated tors', 'tors init guess', 'lower_bound', 'upper_bound'])

In [8]:
print(coefs_noisy.shape)

(60,)


In [8]:
v_grid = (1/2)*(grid[1:]+grid[:-1])
# visu.plot_array_2D_names(v_grid, [curv(v_grid), FS_statespace.theta(v_grid)[:,0], theta_noisy(v_grid)[:,0]], ['curv exact', 'estimated curv', 'curv init guess'])
# visu.plot_array_2D_names(v_grid, [tors(v_grid), FS_statespace.theta(v_grid)[:,1],  theta_noisy(v_grid)[:,1]], ['tors exact', 'estimated tors', 'tors init guess'])
visu.plot_array_2D_names(grid, [curv(grid), FS_statespace.theta(grid)[:,0], theta_noisy(grid)[:,0]], ['curv exact', 'estimated curv', 'curv init guess'])
visu.plot_array_2D_names(grid, [tors(grid), FS_statespace.theta(grid)[:,1],  theta_noisy(grid)[:,1]], ['tors exact', 'estimated tors', 'tors init guess'])
# print("estimated sigma:", np.sqrt(FS_statespace.sigma_square))
print("estimated W:", np.round(FS_statespace.W,decimals=6)) 
print("true W:", W) 
print("estimated P0:", FS_statespace.P0) 

estimated W: [[ 1.e-06 -0.e+00  0.e+00]
 [-0.e+00  1.e-06  0.e+00]
 [ 0.e+00  0.e+00  1.e-06]]
true W: [[1.e-06 0.e+00 0.e+00]
 [0.e+00 1.e-06 0.e+00]
 [0.e+00 0.e+00 1.e-06]]
estimated P0: [[ 6.45742173e-07 -9.91988320e-10  7.77858894e-12 -7.24665006e-11
  -4.43018423e-11 -6.63543822e-10]
 [-9.91988320e-10  9.97293476e-09  1.71077577e-10  1.39371138e-11
   3.35091240e-12  1.49579745e-10]
 [ 7.77858894e-12  1.71077577e-10  4.04496230e-07 -1.66032584e-10
  -6.32100964e-10  1.65973639e-11]
 [-7.24665006e-11  1.39371138e-11 -1.66032584e-10  7.09743018e-09
  -8.45024975e-10 -1.49368096e-10]
 [-4.43018423e-11  3.35091240e-12 -6.32100964e-10 -8.45024975e-10
   8.23304932e-09 -5.40191899e-11]
 [-6.63543822e-10  1.49579745e-10  1.65973639e-11 -1.49368096e-10
  -5.40191899e-11  7.87190875e-09]]


In [13]:
visu.plot_array_2D_names(v_grid, [[FS_statespace.Sigma(v_grid[i])[0,0] for i in range(199)], [FS_statespace.Sigma(v_grid[i])[1,1] for i in range(199)]], ['sigma1', 'sigma2'])

In [12]:
FS_statespace = FrenetStateSpace(grid, Y)
P0 = 0.001**2*np.eye(6)
sigma_1 = 0.02
sigma_2 = 0.02
Sigma = lambda s: np.array([[sigma_1**2, 0], [0, sigma_2**2]])
max_iter = 10
tol = 1e-5 
reg_param_list = np.array([[0.01,0.01], [0.001,0.001], [0.0001,0.0001], [0.00001,0.00001], [0.000001,0.000001]])
tab_results = FS_statespace.expectation_maximization(tol, max_iter, nb_basis=30, regularization_parameter_list=reg_param_list, init_params = {"W":W, "coefs":coefs_noisy, "mu0":mu0, "Sigma":Sigma, "P0":P0}, init_states = None, method='autre', model_Sigma='single_constant')


-------------- Iteration 1 / 10  --------------
___ E step ___
___ M step ___
sigma: 0.01977354845227034
iteration for optimization of coefs and Sigma: 0 , relative error: 0.06197080811589998
sigma: 0.019773520796051495
iteration for optimization of coefs and Sigma: 1 , relative error: 1.0819485295417896e-05
lambda value: [0.01 0.01]  GCV score: 2.9677451898409926 SSE: 399.99999999999994 df: 35.81567620730358
sigma: 0.019772389429555375
iteration for optimization of coefs and Sigma: 0 , relative error: 0.0620346274129071
sigma: 0.019772385177622996
iteration for optimization of coefs and Sigma: 1 , relative error: 3.7612896325984122e-06
lambda value: [0.001 0.001]  GCV score: 4.220640144121074 SSE: 400.00000000000006 df: 62.324763619013524
sigma: 0.019772180565326054
iteration for optimization of coefs and Sigma: 0 , relative error: 0.062043381006608785
sigma: 0.019772182864706073
iteration for optimization of coefs and Sigma: 1 , relative error: 7.783136689395655e-05
lambda value: [0.

In [13]:
v_grid = (1/2)*(grid[1:]+grid[:-1])

""" Plot loglikelihood """
visu.plot_array_2D_names(np.linspace(1,10), [FS_statespace.tab_loglikelihood], ['loglikelihood values'])

""" Plot curvatures and torsions """
visu.plot_array_2D_names(v, [curv(v_grid), theta_kern_noisy(v_grid)[:,0]] + [FS_statespace.tab_theta[i][:,0] for i in range(10)], ['exact param', 'initial guess']+['estimation iteration: '+str(i) for i in range(10)], legend={"index":True, "title":"Results curvature", "x axis":"", "y axis":""})
visu.plot_array_2D_names(v, [tors(v_grid), theta_kern_noisy(v_grid)[:,1]] + [FS_statespace.tab_theta[i][:,1] for i in range(10)], ['exact param', 'initial guess']+['estimation iteration: '+str(i) for i in range(10)], legend={"index":True, "title":"Results torsion", "x axis":"", "y axis":""})

In [20]:
x = np.zeros(200)
print(x.ndim)
print(x.shape)

1
(200,)


In [79]:
from skfda.representation.basis import VectorValued, BSpline
from skfda.misc.regularization import TikhonovRegularization, compute_penalty_matrix
from skfda.misc.operators import LinearDifferentialOperator
list_basis = [BSpline(domain_range=(0,1), n_basis=5, order=4), BSpline(domain_range=(0,1), n_basis=6, order=4)]
vect_basis = VectorValued(list_basis)
vect_pen_mat = compute_penalty_matrix(basis_iterable=(vect_basis,),regularization_parameter=1,regularization=TikhonovRegularization(LinearDifferentialOperator(2)))
basis = list_basis[0]
pen_mat = compute_penalty_matrix(basis_iterable=(basis,),regularization_parameter=1,regularization=TikhonovRegularization(LinearDifferentialOperator(2)))

In [96]:
print((np.squeeze(vect_basis.evaluate(np.array([1,2,3]))).T @ np.zeros(11)).shape)


(2, 3)


In [75]:
print(vect_basis)
print(basis)
print(pen_mat)
print(vect_pen_mat.shape)

VectorValued(basis_list=(BSpline(domain_range=((0, 1),), n_basis=5, order=4, knots=(0.0, 0.5, 1.0)), BSpline(domain_range=((0, 1),), n_basis=6, order=4, knots=(0.0, 0.3333333333333333, 0.6666666666666666, 1.0))))
BSpline(domain_range=((0, 1),), n_basis=5, order=4, knots=(0.0, 0.5, 1.0))
[[  96. -132.   24.   12.    0.]
 [-132.  192.  -48.  -24.   12.]
 [  24.  -48.   48.  -48.   24.]
 [  12.  -24.  -48.  192. -132.]
 [   0.   12.   24. -132.   96.]]
(11, 11)


In [51]:
V = np.expand_dims(np.linspace(0,1,3), 1)
vect_basis_matrix = vect_basis(V,).reshape((vect_basis.n_basis, -1)).T
basis_matrix = basis(V,).reshape((basis.n_basis, -1)).T
print(vect_basis_matrix.shape)
print(basis_matrix.shape)
print(vect_basis_matrix[np.array([0,2,4]),:5])
print(basis_matrix)

(6, 11)
(3, 5)
[[1.   0.   0.   0.   0.  ]
 [0.   0.25 0.5  0.25 0.  ]
 [0.   0.   0.   0.   1.  ]]
[[1.   0.   0.   0.   0.  ]
 [0.   0.25 0.5  0.25 0.  ]
 [0.   0.   0.   0.   1.  ]]


In [55]:
print(np.round(vect_basis_matrix.T @ vect_basis_matrix, decimals=2))
print(np.round(basis_matrix.T @ basis_matrix, decimals=2))

[[1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.06 0.12 0.06 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.12 0.25 0.12 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.06 0.12 0.06 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.01 0.01 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.01 0.22 0.22 0.01 0.  ]
 [0.   0.   0.   0.   0.   0.   0.01 0.22 0.22 0.01 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.01 0.01 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]
[[1.   0.   0.   0.   0.  ]
 [0.   0.06 0.12 0.06 0.  ]
 [0.   0.12 0.25 0.12 0.  ]
 [0.   0.06 0.12 0.06 0.  ]
 [0.   0.   0.   0.   1.  ]]


In [36]:
np.array([[2,0,0],[0,3,0],[0,3,0]])@np.array([[1,0,0],[0,1,1], [0,1,1]])

array([[2, 0, 0],
       [0, 3, 3],
       [0, 3, 3]])

In [37]:
np.diag(np.concatenate((np.repeat(2,1),np.repeat(3,2))))

array([[2, 0, 0],
       [0, 3, 0],
       [0, 0, 3]])

In [74]:
x = np.ones((5,2))
x[:,0] = 2*x[:,0]
print(np.eye(5) @ x)
from scipy.linalg import block_diag
print(block_diag(*[np.diag(x[i]) for i in range(5)]))
print(block_diag(*np.apply_along_axis(np.diag, 1, x)))

[[2. 1.]
 [2. 1.]
 [2. 1.]
 [2. 1.]
 [2. 1.]]
[[2. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
[[2. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [77]:
coefs_tem = np.zeros((10))
coefs_tem = np.reshape(coefs_tem,(-1,2))
print(coefs_tem.shape)
new_coefs = np.reshape(coefs_tem, (-1,))
print(new_coefs.shape)

(5, 2)
(10,)
